# TimeSeries Tutorial

This notebook walks through the `TimeSeries` container used for defining time-dependent control envelopes for QCOM Hamiltonians. It mirrors the style of the AtomRegister tutorial.

**Contents**

1) [Setup](#setup)  
2) [Create an empty series](#create)  
3) [Add points and view channels](#add)  
4) [Add a batch (add_series)](#batch)  
5) [Read-only views & domains](#views)  
6) [Interpolation with value_at](#interp)  
7) [Editing: remove & clear](#edit)  
8) [Plotting the envelopes](#plot)  
9) [Normalized mode](#normalized)  


## 1) Setup  <a id='setup'></a>

Most users will install QCOM via `pip` and import directly from the package. Make sure QCOM is installed:

```bash
pip install qcom
```

Then import the class from the submodule for explicit, future-proof imports:
```python
from qcom.time_series import TimeSeries
import numpy as np
import matplotlib.pyplot as plt
```

> **Why this import?**  
> Importing from the submodule (e.g., `qcom.time_series`) is explicit and resilient if top-level re-exports change later.

In [ ]:
from qcom.time_series import TimeSeries
import numpy as np
import matplotlib.pyplot as plt

## 2) Create an empty series  <a id='create'></a>

You can start with an **empty** `TimeSeries` and build it up incrementally. Calling `len(series)` returns the number of channels currently stored.

In [ ]:
# Start empty (absolute mode by default)
ts = TimeSeries()
print(ts)
print('Number of channels:', len(ts))

## 3) Add points and view channels  <a id='add'></a>

Use `add_point(name, t, value)` to insert a single sample. Times in each channel must be **strictly increasing**. If you add a time that already exists (within a small tolerance), the value is **replaced** rather than duplicated.

In [ ]:
# Add a few Omega points (units depend on mode; here absolute)
ts.add_point('Omega', 0.0, 0.0)
ts.add_point('Omega', 1e-6, 2.0e6)
ts.add_point('Omega', 2e-6, 0.0)

# Add a couple of Delta points (e.g., rad/s)
ts.add_point('Delta', 0.0, -1.0e6)
ts.add_point('Delta', 2e-6,  1.0e6)

print(ts)
print('Channel names:', ts.channel_names)

# You can inspect read-only views of times/values
print('Omega times  :', ts.times('Omega'))
print('Omega values :', ts.values('Omega'))

## 4) Add a batch (`add_series`)  <a id='batch'></a>

`add_series(name, times, values)` merges a batch, sorts by time, collapses duplicates (last-wins), and merges with any existing samples for that channel.

In [ ]:
# Add a batch to Phi
t_phi = [0.0, 0.5e-6, 1.5e-6, 2.0e-6]
v_phi = [0.0, 0.25*np.pi, 0.75*np.pi, np.pi]
ts.add_series('Phi', t_phi, v_phi)
print(ts)
print('Phi times  :', ts.times('Phi'))
print('Phi values :', ts.values('Phi'))

## 5) Read-only views & domains  <a id='views'></a>

- `.times(name)` / `.values(name)` return **read-only views**.
- `.channels` returns a **read-only mapping** of all channels to their (times, values) read-only arrays.
- `.domain(name)` returns `(t_min, t_max)` for that channel; `.domain()` with no name returns the **union domain** across all channels.

In [ ]:
print('Union domain:', ts.domain())
print('Delta domain:', ts.domain('Delta'))

ch = ts.channels
print('Channels keys:', list(ch.keys()))
print('Omega (view) types:', type(ch['Omega'][0]), type(ch['Omega'][1]))

## 6) Interpolation with `value_at`  <a id='interp'></a>

- Linear interpolation within each channel's domain.
- Outside a channel's domain (before first or after last sample), the value is **0.0**.
- If you request a channel that doesn't exist, you get an all-zeros array.

You can query multiple channels at once, or use `value_at_channel(name, tq)` for convenience.

In [ ]:
tq = np.linspace(-0.5e-6, 2.5e-6, 9)  # includes times outside the domain
vals = ts.value_at(tq, channels=['Omega','Delta','Phi','NotAChannel'])
for k, arr in vals.items():
    print(f"{k:12s} -> {np.array_str(arr, precision=3, suppress_small=False)}")

## 7) Editing: remove & clear  <a id='edit'></a>

- `remove_point(name, t)` removes the sample at time `t` (within tolerance).  
- `remove_at(name, index)` removes by integer index.  
- `clear_channel(name)` deletes an entire channel.  
- `clear()` deletes **all** channels.

In [ ]:
# Remove the middle Omega sample by its time
ts.remove_point('Omega', 1e-6)
print('Omega after remove_point:', ts.times('Omega'), ts.values('Omega'))

# Remove the last Delta sample by index
ts.remove_at('Delta', len(ts.times('Delta')) - 1)
print('Delta after remove_at   :', ts.times('Delta'), ts.values('Delta'))

## 8) Plotting the envelopes  <a id='plot'></a>

The `plot()` method draws three stacked subplots (Ω, Δ, Φ) with shared time axis.  
If a channel is missing, the subplot is annotated as "not provided (defaults to 0)".

> **Note:** The figure is **not** automatically shown. Call `plt.show()` to display it, or `fig.savefig(...)` to save.

In [ ]:
fig, axes = ts.plot(figsize=(7, 5))
plt.show()

## 9) Normalized mode  <a id='normalized'></a>

In **normalized** mode, the container enforces ranges:
- $\Omega_{\text{env}} \in [0,1]$  
- $\Delta_{\text{env}} \in [-1,1]$  
- $\Phi_{\text{env}}$ is unbounded.

This is useful when you want to apply site-dependent scaling at evolve time (e.g., $\Omega_i(t) = \Omega_{\max, i} \cdot \Omega_{\text{env}}(t)$).

In [ ]:
# Example: normalized mode
tsn = TimeSeries(mode='normalized')
tsn.add_series('Omega', [0.0, 1.0, 2.0], [0.0, 1.0, 0.0])  # triangle in [0,1]
tsn.add_series('Delta', [0.0, 2.0], [-1.0, 1.0])           # sweep in [-1,1]
print(tsn)

# Interpolate
tq = np.linspace(0.0, 2.0, 5)
vals = tsn.value_at(tq, channels=['Omega','Delta'])
for k, arr in vals.items():
    print(f"{k:12s} -> {np.array_str(arr, precision=3)}")

# Plot (y-lims reflect normalized ranges for Ω and Δ)
fig, axes = tsn.plot(figsize=(7, 5))
plt.show()